# Import 


In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q  http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"

spark-2.4.0-bin-hadoop2.7/
spark-2.4.0-bin-hadoop2.7/python/
spark-2.4.0-bin-hadoop2.7/python/setup.cfg
spark-2.4.0-bin-hadoop2.7/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/resultiterable.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/shell.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/heapq3.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/join.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/version.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/rdd.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/java_gateway.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/find_spark_home.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/_globals.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/worker.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/accumulators.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/feature.py
spark-2.4.0-bin-hadoop2.7/python/pyspark

In [0]:
import findspark
findspark.init()

In [0]:
import pyspark
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

In [0]:
sc = SparkContext()

In [0]:
spark = SparkSession(sc)

# Workplace

In [0]:
import numpy as np 
import pandas as pd

In [0]:
true_path = "/content/drive/My Drive/Big Data/LDS9_LeNguyenThanhDuyen/LDS9_DeThi/LDS9_DeThi/Du lieu cung cap/fake-and-real-news-dataset/True.csv"
fake_path = "/content/drive/My Drive/Big Data/LDS9_LeNguyenThanhDuyen/LDS9_DeThi/LDS9_DeThi/Du lieu cung cap/fake-and-real-news-dataset/Fake.csv"

In [0]:
true_data = spark.read.csv(true_path, header=True, inferSchema = True)
fake_data = spark.read.csv(fake_path, header=True, inferSchema = True)

In [10]:
true_data.count()

21417

In [11]:
# Number of news fake
fake_data.count()

23489

In [12]:
true_data.show(5)

+--------------------+--------------------+------------+------------------+
|               title|                text|     subject|              date|
+--------------------+--------------------+------------+------------------+
|As U.S. budget fi...|WASHINGTON (Reute...|politicsNews|December 31, 2017 |
|U.S. military to ...|WASHINGTON (Reute...|politicsNews|December 29, 2017 |
|Senior U.S. Repub...|WASHINGTON (Reute...|politicsNews|December 31, 2017 |
|FBI Russia probe ...|WASHINGTON (Reute...|politicsNews|December 30, 2017 |
|Trump wants Posta...|SEATTLE/WASHINGTO...|politicsNews|December 29, 2017 |
+--------------------+--------------------+------------+------------------+
only showing top 5 rows



In [13]:
fake_data.show(5)

+--------------------+--------------------+-------+-----------------+
|               title|                text|subject|             date|
+--------------------+--------------------+-------+-----------------+
| Donald Trump Sen...|Donald Trump just...|   News|December 31, 2017|
| Drunk Bragging T...|House Intelligenc...|   News|December 31, 2017|
| Sheriff David Cl...|On Friday, it was...|   News|December 30, 2017|
| Trump Is So Obse...|On Christmas day,...|   News|December 29, 2017|
| Pope Francis Jus...|Pope Francis used...|   News|December 25, 2017|
+--------------------+--------------------+-------+-----------------+
only showing top 5 rows



In [0]:
from pyspark.sql.functions import lit, rand
data = true_data.withColumn('fake', lit(0)).union(fake_data.withColumn('fake', lit(1))).orderBy(rand())

In [16]:
data.groupBy('fake').count().show()

+----+-----+
|fake|count|
+----+-----+
|   1|23489|
|   0|21417|
+----+-----+



In [24]:
data.show(5, False)

+-----------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
# Check null
from pyspark.sql.functions import col
from pyspark.sql.functions import isnull, when, count, col, isnan

In [19]:
data_entry = data.count()
null_data = data.select([count(when(col(c).isNull(),c)).alias(c) for c in data.columns]).toPandas().T
null_data["percentage"] = (null_data[0]/data_entry)*100
null_data

,0,percentage
title,0,0.000000
text,8,0.017815
subject,8,0.017815
date,8,0.017815
fake,0,0.000000


In [0]:
data = data.na.drop()

# Pre processing

In [21]:
!pip install contractions
import contractions

     |████████████████████████████████| 317kB 3.8MB/s 
     |████████████████████████████████| 245kB 35.6MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81700 sha256=84048c7d6a1c7d9bdcacd0320d70af646d35c1396cabbd36ab309110f04d7e72
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [0]:
from pyspark.sql.functions import col, udf

In [0]:
exp_con = udf (lambda x: contractions.fix(x))

In [0]:
data_sub = data.withColumn('title_exp', exp_con(col('title')))

In [26]:
data_sub.show(5, False)

+-----------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
data_sub = data_sub.withColumn('text_exp', exp_con(col('text')))

In [28]:
data_sub.show(5)

+--------------------+--------------------+---------+-----------------+----+--------------------+--------------------+
|               title|                text|  subject|             date|fake|           title_exp|            text_exp|
+--------------------+--------------------+---------+-----------------+----+--------------------+--------------------+
| Hispanic Rep Jus...|Now that Republic...|     News|     June 6, 2016|   1| Hispanic Rep Jus...|Now that Republic...|
|Fraud probe to de...|MONROVIA (Reuters...|worldnews|November 3, 2017 |   0|Fraud probe to de...|MONROVIA (Reuters...|
|HYPOCRITES! CHECK...|It s interesting ...| politics|     Jul 24, 2016|   1|HYPOCRITES! CHECK...|It s interesting ...|
|Italian man sente...|ROME (Reuters) - ...|worldnews|October 27, 2017 |   0|Italian man sente...|ROME (Reuters) - ...|
| Fox Host: Anders...|There is an attit...|     News|    June 19, 2016|   1| Fox Host: Anders...|There is an attit...|
+--------------------+--------------------+-----

In [0]:
from pyspark.sql.functions import *
data_sub = data_sub.withColumn("length", length(col("text_exp")))

# Feature transformation

In [0]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, RegexTokenizer
from pyspark.ml.feature import CountVectorizer, IDF, StringIndexer, NGram

#tokenizer1 = Tokenizer(inputCol = "Title", outputCol = "title_token")
#tokenizer2 = Tokenizer(inputCol = "Review Text", outputCol = "review_token")

tokenizer1 = RegexTokenizer(inputCol = "title_exp", outputCol = "title_token", pattern="\\W")
tokenizer2 = RegexTokenizer(inputCol = "text_exp", outputCol = "text_token", pattern="\\W")


stopremove1 = StopWordsRemover(inputCol = "title_token", outputCol = "title_stop_tokens")
stopremove2 = StopWordsRemover(inputCol = "text_token", outputCol = "text_stop_tokens")

count_vec1 = CountVectorizer(inputCol = "title_stop_tokens", outputCol = "c_vec1")
count_vec2 = CountVectorizer(inputCol = "text_stop_tokens", outputCol = "c_vec2")

idf1 = IDF(inputCol = "c_vec1", outputCol = "tf_idf1")
idf2 = IDF(inputCol = "c_vec2",outputCol = "tf_idf2")

subject_to_num = StringIndexer(inputCol= 'subject', outputCol='subject_idx')
fake_to_num = StringIndexer(inputCol= 'fake', outputCol='label')

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [0]:
clean_up = VectorAssembler(inputCols=["tf_idf1", "tf_idf2","length", 'subject_idx'], outputCol='features')

# Pipeline

In [0]:
from pyspark.ml import Pipeline

In [0]:
data_prep_pipe = Pipeline(stages=[subject_to_num,fake_to_num,tokenizer1,tokenizer2,stopremove1, stopremove2 ,count_vec1,count_vec2,idf1,idf2,clean_up])

In [0]:
cleaner = data_prep_pipe.fit(data_sub)

In [0]:
clean_data = cleaner.transform(data_sub)

In [40]:
clean_data.show(5)

+--------------------+--------------------+---------+-----------------+----+--------------------+--------------------+------+-----------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|                text|  subject|             date|fake|           title_exp|            text_exp|length|subject_idx|label|         title_token|          text_token|   title_stop_tokens|    text_stop_tokens|              c_vec1|              c_vec2|             tf_idf1|             tf_idf2|            features|
+--------------------+--------------------+---------+-----------------+----+--------------------+--------------------+------+-----------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|

# Train test split

In [0]:
clean_data = clean_data.select(['label','features'])

In [42]:
clean_data.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(138934,[0,112,19...|
|  1.0|(138934,[16,95,25...|
|  0.0|(138934,[47,216,3...|
|  1.0|(138934,[72,86,18...|
|  0.0|(138934,[1,73,130...|
+-----+--------------------+
only showing top 5 rows



In [43]:
clean_data.groupby('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|23481|
|  1.0|21417|
+-----+-----+



In [0]:
training, testing = clean_data.randomSplit([0.8,0.2])

In [45]:
training.groupby('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|18814|
|  1.0|17122|
+-----+-----+



In [46]:
testing.groupby('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0| 4667|
|  1.0| 4295|
+-----+-----+



# Model

In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
lg = LogisticRegression(maxIter=40, #45 0.6267 #40 6281 
                        regParam=0.01, #
                        elasticNetParam=0.3, #0.3 62.6
                        )

In [0]:
predictor = lg.fit(training)

In [0]:
test_results = predictor.transform(testing)

In [55]:
acc_eval = MulticlassClassificationEvaluator()
acc= acc_eval.evaluate(test_results)
acc

0.9952025211364806

In [56]:
test_results.show(10)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(138934,[0,1,61,4...|[5.52741911882043...|[0.99603950979618...|       0.0|
|  0.0|(138934,[0,2,5,9,...|[1.39285881684346...|[0.80104824480453...|       0.0|
|  0.0|(138934,[0,9,64,3...|[5.75441929015854...|[0.99684126303676...|       0.0|
|  0.0|(138934,[0,11,181...|[4.19793381233517...|[0.98519586320392...|       0.0|
|  0.0|(138934,[0,41,81,...|[3.57885248439416...|[0.97284999003855...|       0.0|
|  0.0|(138934,[0,45,268...|[5.81071470675785...|[0.99701365640661...|       0.0|
|  0.0|(138934,[0,167,18...|[4.94484196621348...|[0.99293029678612...|       0.0|
|  0.0|(138934,[1,37,115...|[5.46003075723345...|[0.99576459007888...|       0.0|
|  0.0|(138934,[1,58,194...|[5.08445805521319...|[0.99384586585373...|       0.0|
|  0.0|(138934,[

In [0]:
predict_n_label = test_results.select(["label", "prediction"])

In [0]:
from pyspark.mllib.evaluation import MulticlassMetrics

In [0]:
metrics = MulticlassMetrics(predict_n_label.rdd.map(tuple))

In [0]:
pd.set_option('display.max_columns', 30)

In [0]:
matrix = pd.DataFrame(metrics.confusionMatrix().toArray())

In [63]:
matrix

,0,1
0,4632.0,8.0
1,35.0,4287.0
